# Project: Multiclass Classification Of Flower Species
In this project tutorial you will discover how you can use Keras to develop and evaluate neural
network models for multiclass classification problems. After completing this step-by-step tutorial,
you will know:
1. How to load data from CSV and make it available to Keras.
2. How to prepare multiclass classification data for modeling with neural networks.
3. How to evaluate Keras neural network models with scikit-learn.
Let’s get started.

### 1.1 Iris Flowers Classification Dataset
In this tutorial we will use the standard machine learning problem called the iris flowers dataset.
This dataset is well studied and is a good problem for practicing on neural networks because
all of the 4 input variables are numeric and have the same scale in centimeters. Each instance
describes the properties of an observed flower measurements and the output variable is specific
iris species. The attributes for this dataset can be summarized as follows:
1. Sepal length in centimeters.
2. Sepal width in centimeters.
3. Petal length in centimeters.
4. Petal width in centimeters.
5. Class.

This is a multiclass classification problem, meaning that there are more than two classes
to be predicted, in fact there are three flower species. This is an important type of problem
on which to practice with neural networks because the three class values require specialized
handling. Below is a sample of the first five of the 150 instances:

```5.1,3.5,1.4,0.2,Iris-setosa
4.9,3.0,1.4,0.2,Iris-setosa
4.7,3.2,1.3,0.2,Iris-setosa
4.6,3.1,1.5,0.2,Iris-setosa
5.0,3.6,1.4,0.2,Iris-setosa```

Sample of the Iris Flowers Dataset.

The iris flower dataset is a well studied problem and a such we can expect to achieve a model
accuracy in the range of 95% to 97%. This provides a good target to aim for when developing
our models. The dataset is provided in the bundle of sample code provided with this book. You
can also download the iris flowers dataset from the UCI Machine Learning repository and place
it in your current working directory with the filename iris.csv. You can learn more about the
iris flower classification dataset on the UCI Machine Learning Repository page.

#### 1.2 Import Classes and Functions
We can begin by importing all of the classes and functions we will need in this tutorial. This
includes both the functionality we require from Keras, but also data loading from Pandas as
well as data preparation and model evaluation from scikit-learn.


In [1]:
# Import Classes and Functions.
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

Using TensorFlow backend.
/home/isaac/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


#### 1.3 Initialize Random Number Generator
Next we need to initialize the random number generator to a constant value. This is important
to ensure that the results we achieve from this model can be achieved again precisely. It ensures
that the stochastic process of training a neural network model can be reproduced.

In [3]:
# fix random seed for reproducibility
# Initialize Random Number Generator.
seed = 7
numpy.random.seed(seed)

### 1.4 Load The Dataset
The dataset can be loaded directly. Because the output variable contains strings, it is easiest to
load the data using pandas. We can then split the attributes (columns) into input variables (X)
and output variables (Y ).

In [10]:
# load dataset
# Load Dataset And Separate Into Input and Output Variables.
dataframe = pandas.read_csv("iris.csv", header=None)
dataset = dataframe.values
X = dataset[:,0:4].astype(float)
Y = dataset[:,4]

### 1.5 Encode The Output Variable
The output variable contains three different string values. When modeling multiclass classification
problems using neural networks, it is good practice to reshape the output attribute from a
vector that contains values for each class value to be a matrix with a boolean for each class
value and whether or not a given instance has that class value or not. This is called one hot
encoding or creating dummy variables from a categorical variable. For example, in this problem
the three class values are Iris-setosa, Iris-versicolor and Iris-virginica. If we had the
three observations:
Iris-setosa
Iris-versicolor
Iris-virginica
Listing 10.5: Three Classes In The Iris Dataset.
We can turn this into a one-hot encoded binary matrix for each data instance that would
look as follows:
```
Iris-setosa, Iris-versicolor, Iris-virginica
1,     0,     0
0,     1,     0
0,     0,     1```

Listing 10.6: One Hot Encoding of The Classes In The Iris Dataset.
We can do this by first encoding the strings consistently to integers using the scikit-learn
class LabelEncoder. Then convert the vector of integers to a one hot encoding using the Keras
function to categorical().

In [11]:
# encode class values as integers
# One Hot Encoding Of Iris Dataset Output Variable.
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

### 1.6 Define The Neural Network Model
The Keras library provides wrapper classes to allow you to use neural network models developed
with Keras in scikit-learn as we saw in the previous lesson. There is a KerasClassifier class
in Keras that can be used as an Estimator in scikit-learn, the base type of model in the library.
The KerasClassifier takes the name of a function as an argument. This function must return
the constructed neural network model, ready for training.
Below is a function that will create a baseline neural network for the iris classification
problem. It creates a simple fully connected network with one hidden layer that contains 4
neurons, the same number of inputs (it could be any number of neurons). The hidden layer uses
a rectifier activation function which is a good practice. Because we used a one-hot encoding for
our iris dataset, the output layer must create 3 output values, one for each class. The output
value with the largest value will be taken as the class predicted by the model. The network
topology of this simple one-layer neural network can be summarized as:
```
4 inputs -> [4 hidden nodes] -> 3 outputs```

Note that we use a sigmoid activation function in the output layer. This is to ensure the
output values are in the range of 0 and 1 and may be used as predicted probabilities. Finally,
the network uses the efficient ADAM gradient descent optimization algorithm with a logarithmic
loss function, which is called categorical crossentropy in Keras.


In [12]:
# define baseline model
# Define and Compile the Neural Network Model.
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(4, input_dim=4, init= 'normal' , activation= 'relu' ))
    model.add(Dense(3, init= 'normal', activation= 'sigmoid' ))
    # Compile model
    ## CODE HERE ##
    
    return model

We can now create our KerasClassifier for use in scikit-learn. We can also pass arguments
in the construction of the KerasClassifier class that will be passed on to the fit() function
internally used to train the neural network. Here, we pass the number of epochs nb epoch as
200 and batch size as 5 to use when training the model. Debugging is also turned off when
training by setting verbose to 0.

In [13]:
# Create Wrapper For Neural Network Model For Use in scikit-learn.
## CODE HERE ##


### 1.7 Evaluate The Model with k-Fold Cross Validation
We can now evaluate the neural network model on our training data. The scikit-learn library
has excellent capability to evaluate models using a suite of techniques. The gold standard for
evaluating machine learning models is k-fold cross validation. First we can define the model evaluation procedure. Here, we set the number of folds to be 10 (an excellent default) and to shuffle the data before partitioning it.

In [14]:
#Prepare Cross Validation.
kfold = KFold(n=len(X), n_folds=10, shuffle=True, random_state=seed)


Now we can evaluate our model (estimator) on our dataset (X and dummy y) using a 10-fold
cross validation procedure (kfold). Evaluating the model only takes approximately 10 seconds
and returns an object that describes the evaluation of the 10 constructed models for each of the
splits of the dataset.

In [16]:
# Evaluate the Neural Network Model.
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Accuracy: 95.33% (4.27%)


The results are summarized as both the mean and standard deviation of the model accuracy
on the dataset. This is a reasonable estimation of the performance of the model on unseen data.
It is also within the realm of known top results for this problem.

Baseline: 95.33% (4.27%)
###### Estimated Accuracy of Neural Network Model on the Iris Dataset.

### 1.8 Summary
In this lesson you discovered how to develop and evaluate a neural network using the Keras
Python library for deep learning. By completing this tutorial, you learned:
1. How to load data and make it available to Keras.
2. How to prepare multiclass classification data for modeling using one hot encoding.
3. How to use Keras neural network models with scikit-learn.
4. How to define a neural network using Keras for multiclass classification.
5. How to evaluate a Keras neural network model using scikit-learn with k-fold cross validation.

## Next
This was your first end-to-end project using Keras on a standalone dataset. In the next tutorial
you will develop neural network models for a binary classification problem and tune them to get
increases in model performance.